# Config

In [55]:
print('Hello world')

Hello world


In [56]:
import pandas as pd
import numpy as np
import spacy
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import re


SPACY_MODEL = 'en_core_web_sm'
SBERT_MODEL = 'all-mpnet-base-v2'


In [57]:
job_offers = {
    'poor': {
        'text': """JOB OFFER: Certified Commercial Plumbing Technician

We are seeking a reliable and physically capable Certified Commercial Plumbing Technician to join our field service team.
Required Technical Skills:
- Current state certification in commercial plumbing installation and repair.
- Expertise in using and maintaining standard plumbing tools (e.g., pipe cutters, soldering torches).
- Proven experience with various piping materials (e.g., copper, PVC, PEX).
- Ability to read and interpret construction blueprints for pipe routing and material specifications.

Soft Skills & Team Requirements:
- A strong sense of **physical endurance** and comfort working in confined spaces.
- Excellent **time management** and ability to meet strict project deadlines.
- **Customer service focus** for interacting with clients on-site.""",
        'score': '0-10'
    },
    'medium': {
        'text': """JOB OFFER: Junior Front-End Web Developer

We are looking for a creative Junior Front-End Web Developer to assist in building and maintaining our client-facing applications.
Required Technical Skills:
- Proficiency in core web technologies: **HTML5, CSS3, and JavaScript**.
- Familiarity with a modern front-end framework (e.g., React or Vue.js).
- Basic experience querying data via APIs and managing simple back-end data using **SQL**.
- Knowledge of UI/UX principles and responsive design best practices.

Soft Skills & Team Requirements:
- A strong sense of **design aesthetics** and visual appeal.
- Proven ability to **collaborate effectively** with design and back-end teams.
- **Desire for knowledge** of emerging web standards.""",
        'score': '40-60'
    },
    'perfect': {
        'text': """JOB OFFER: Data Science Intern / Junior Data Analyst

We are seeking an enthusiastic and technically proficient graduate to join our Data Science team for a junior role or internship.
Required Technical Skills:
- **Expert level coding** proficiency in **Python** for data manipulation and analysis.
- Deep knowledge of relational databases, with proven experience managing data using **Structured Query Language (SQL)**.
- Hands-on experience with specific database environments, preferably **Postgres**.
- Successful application of **statistical methods** for generating business insights.

Soft Skills & Team Requirements:
- Proven ability to **work together effectively** in cross-functional teams.
- Highly **articulate** and able to clearly explain complex technical results to business stakeholders.
- Driven by a **desire for knowledge** and continuous learning within the Data Science domain.""",
        'score': '90-100'
    }
}

In [58]:
cv_candidate = """
CANDIDATE PROFILE: Data Science Graduate

Summary: Enthusiastic graduate with a passion for transforming complex data into actionable insights and analysing).
Technical Expertise:
- **Expert level coding** in Python, used for ETL and complex calculations.
- Deep knowledge of relational databases, managing data using **Structured Query Language (SQL)**.
- Hands-on experience with **Postgres**.
- Successfully applied **statistical methods** in university projects.

Personal and Team Skills:
- Proven ability to **work together effectively** in cross-functional teams.
- Highly **articulate** and able to clearly explain technical results to non-technical stakeholders.
- Driven by a **desire for knowledge** and continuous improvement.
"""

# Transform

In [59]:
from spacy import Language
from spacy.tokens import Doc
from typing import List, Tuple, Dict
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix

## Transform for TF-IDF

In [60]:
def set_up_obj():
    try:
        nlp_ = spacy.load(SPACY_MODEL)
    except OSError:
        raise Exception(f'Model {SPACY_MODEL} is not installed, use python -m spacy download')

    tfidf_vectorizer = TfidfVectorizer(
        stop_words='english',
        # ngram_range=(1, 2),
        # min_df=2,
    )
    return nlp_, tfidf_vectorizer


In [61]:
def get_lemmas_from_doc(doc: Doc) -> List[str]:
    return [token.lemma_.lower() for token in doc if token.is_alpha]

In [62]:
def lemmatize_and_clean_texts(texts: List[str], nlp: Language) -> List[str]:
    processed_texts = []

    for doc in nlp.pipe(texts):
        lemmas_list = get_lemmas_from_doc(doc)
        processed_texts.append(' '.join(lemmas_list))

    return processed_texts


In [63]:
def vectorize_lemmas(processed_texts: List[str], vectorizer: TfidfVectorizer) -> csr_matrix:
    return vectorizer.fit_transform(processed_texts)

## Transform for SBERT

In [64]:
def set_up_obj_sbert():
    try:
        nlp = spacy.load(SPACY_MODEL)
    except OSError:
        raise Exception(f'Model {SPACY_MODEL} is not installed, use python -m spacy download')
    return nlp

In [65]:
def clean_texts_for_sbert(job_offer: str, cv: str, nlp: Language,
                          window_size: int = 3, overlap: int = 1) -> Dict[str, List[str]]:
    all_chunks = {'job_offer': [], 'cv': []}

    for key, text in zip(('job_offer', 'cv'), (job_offer, cv)):
        
        if not isinstance(text, str):
            continue

        flat_text = re.sub(r'\s+', ' ', text).strip()
        doc = nlp(flat_text)
        sentences = [sent.text.strip() for sent in doc.sents]

        if len(sentences) <= window_size:
            all_chunks[key].append(flat_text)
            continue

        step = window_size - overlap
        if step < 1: step = 1

        for i in range(0, len(sentences), step):
            group = sentences[i:i + window_size]
            chunk_text = ' '.join(group)
            all_chunks[key].append(chunk_text)

    return all_chunks

In [66]:
sbert_text_clean_poor = clean_texts_for_sbert(
    job_offers['poor']['text'], cv_candidate, set_up_obj_sbert())
sbert_text_clean_medium = clean_texts_for_sbert(
    job_offers['medium']['text'], cv_candidate, set_up_obj_sbert())
sbert_text_clean_perfect = clean_texts_for_sbert(
    job_offers['perfect']['text'], cv_candidate, set_up_obj_sbert())

In [67]:
sbert_text_clean_poor['job_offer'][0]

'JOB OFFER: Certified Commercial Plumbing Technician We are seeking a reliable and physically capable Certified Commercial Plumbing Technician to join our field service team. Required Technical Skills: - Current state certification in commercial plumbing installation and repair.'

In [68]:
job_offers['poor']['text'][:51]

'JOB OFFER: Certified Commercial Plumbing Technician'

# Calculations

## TF-IDF

In [69]:
from sklearn.metrics.pairwise import cosine_similarity


def tf_idf_pipeline(job_offer: str, cv: str) -> Tuple[csr_matrix, TfidfVectorizer]:
    nlp, tfidf_vectorizer = set_up_obj()
    clean_texts = lemmatize_and_clean_texts([job_offer, cv], nlp)
    return vectorize_lemmas(clean_texts, tfidf_vectorizer), tfidf_vectorizer

In [70]:
matrix_poor, tfidf_vec_poor = tf_idf_pipeline(job_offers['poor']['text'], cv_candidate)
matrix_medium, tfidf_vec_medium = tf_idf_pipeline(job_offers['medium']['text'], cv_candidate)
matrix_perfect, tfidf_vec_perfect = tf_idf_pipeline(job_offers['perfect']['text'], cv_candidate)

In [71]:
similarity_poor = cosine_similarity(
    matrix_poor[0:1],
    matrix_poor[1:2]
)
print(f'Similarity TF-IDF poor (Lemmatized, no N-grams): {similarity_poor[0][0] * 100:.2f}%')
print('Score expected:', job_offers['poor']['score'])

Similarity TF-IDF poor (Lemmatized, no N-grams): 10.26%
Score expected: 0-10


In [72]:
similarity_medium = cosine_similarity(
    matrix_medium[0:1],
    matrix_medium[1:2]
)
print(f'Similarity TF-IDF medium (Lemmatized, no N-grams): {similarity_medium[0][0] * 100:.2f}%')
print('Score expected:', job_offers['medium']['score'])

Similarity TF-IDF medium (Lemmatized, no N-grams): 14.84%
Score expected: 40-60


In [73]:
similarity_perfect = cosine_similarity(
    matrix_perfect[0:1],
    matrix_perfect[1:2]
)
print(f'Similarity TF-IDF perfect (Lemmatized, no N-grams): {similarity_perfect[0][0] * 100:.2f}%')
print('Score expected:', job_offers['perfect']['score'])

Similarity TF-IDF perfect (Lemmatized, no N-grams): 56.40%
Score expected: 90-100


In [74]:
def explain_tf_idf(tfidf_vectorizer: TfidfVectorizer, matrix: csr_matrix):
    feature_names = tfidf_vectorizer.get_feature_names_out()
    dense = matrix.todense()

    denselist = dense.tolist()
    df = pd.DataFrame(denselist, columns=feature_names, index=['job_offer', 'cv_candidate'])

    common_words = df.loc['job_offer'] * df.loc['cv_candidate']
    common_words = common_words[common_words > 0].sort_values(ascending=False)

    return common_words



In [75]:
common_words_poor = explain_tf_idf(tfidf_vec_poor, matrix_poor)
print('\nCommon words TF-IDF poor (Lemmatized, no N-grams):')
print(common_words_poor)


Common words TF-IDF poor (Lemmatized, no N-grams):
team          0.022796
technical     0.017097
skills        0.011398
ability       0.011398
use           0.011398
experience    0.005699
prove         0.005699
project       0.005699
expertise     0.005699
work          0.005699
dtype: float64


In [76]:
common_words_medium = explain_tf_idf(tfidf_vec_medium, matrix_medium)
print('\nCommon words TF-IDF medium (Lemmatized, no N-grams):')
print(common_words_medium)


Common words TF-IDF medium (Lemmatized, no N-grams):
datum          0.022836
team           0.022836
knowledge      0.022836
technical      0.017127
skills         0.011418
use            0.011418
effectively    0.005709
desire         0.005709
ability        0.005709
experience     0.005709
manage         0.005709
query          0.005709
sql            0.005709
dtype: float64


In [77]:
common_words_perfect = explain_tf_idf(tfidf_vec_perfect, matrix_perfect)
print('\nCommon words TF-IDF perfect (Lemmatized, no N-grams):')
print(common_words_perfect)


Common words TF-IDF perfect (Lemmatized, no N-grams):
technical       0.046353
team            0.046353
knowledge       0.030902
datum           0.030902
data            0.030902
science         0.023177
graduate        0.015451
experience      0.015451
prove           0.015451
skills          0.015451
database        0.015451
complex         0.015451
use             0.015451
desire          0.007726
deep            0.007726
continuous      0.007726
cross           0.007726
able            0.007726
articulate      0.007726
clearly         0.007726
cod             0.007726
ability         0.007726
hands           0.007726
functional      0.007726
effectively     0.007726
enthusiastic    0.007726
expert          0.007726
explain         0.007726
drive           0.007726
method          0.007726
manage          0.007726
level           0.007726
language        0.007726
insight         0.007726
highly          0.007726
query           0.007726
postgre         0.007726
relational      0.00

## SBERT

In [78]:
def get_match_status(score: float) -> str:
    """Pomocnicza funkcja do etykietowania"""
    if score > 0.75: return "Świetne dopasowanie ✅"
    if score > 0.55: return "Dobre dopasowanie ⚠️"
    if score > 0.35: return "Słabe dopasowanie 🔸"
    return "Brak dopasowania ❌"

In [79]:
def calculate_precise_match(chunks_data: Dict[str, List[str]]) -> Dict[str, any]:
    model = SentenceTransformer(SBERT_MODEL)
    job_offer_chunks = chunks_data['job_offer']
    cv_chunks = chunks_data['cv']

    job_offer_embeddings = model.encode(job_offer_chunks)
    cv_embeddings = model.encode(cv_chunks)

    similarity_scores = cosine_similarity(job_offer_embeddings, cv_embeddings)

    explanation_details = []
    for i, job_offer_chunk in enumerate(job_offer_chunks):
        row_scores = similarity_scores[i]
        max_score = np.max(row_scores)
        best_match_index = np.argmax(row_scores)
        matching_cv_text = cv_chunks[best_match_index]
        explanation_details.append({
            'requirement': job_offer_chunk,
            'best_match_in_cv': matching_cv_text,
            'score': float(max_score),
            'status': get_match_status(max_score)
        })

    overall_similarity = float(np.mean([item['score'] for item in explanation_details]))

    return {
        "overall_score": round(overall_similarity, 4),
        "breakdown": explanation_details,
    }

In [80]:
def sbert_pipeline(job_offer: str, cv: str) -> any:
    chunks_data = clean_texts_for_sbert(job_offer, cv, set_up_obj_sbert())
    report = calculate_precise_match(chunks_data)
    return report

In [81]:
sbert_score_poor = sbert_pipeline(job_offers['poor']['text'], cv_candidate)
print('\n=== POOR MATCH REPORT ===')
print(f"Overall Score: {sbert_score_poor['overall_score']:.4f}")
print(f"Expected Score Range: {job_offers['poor']['score']}")
print("\nBreakdown:")
for i, item in enumerate(sbert_score_poor['breakdown'], 1):
    print(f"\n{i}. Requirement: {item['requirement'][:80]}...")
    print(f"   Best Match: {item['best_match_in_cv'][:80]}...")
    print(f"   Score: {item['score']:.4f}")
    print(f"   Status: {item['status']}")


=== POOR MATCH REPORT ===
Overall Score: 0.4664
Expected Score Range: 0-10

Breakdown:

1. Requirement: JOB OFFER: Certified Commercial Plumbing Technician We are seeking a reliable an...
   Best Match: Personal and Team Skills: - Proven ability to **work together effectively** in c...
   Score: 0.3538
   Status: Słabe dopasowanie 🔸

2. Requirement: Required Technical Skills: - Current state certification in commercial plumbing ...
   Best Match: Personal and Team Skills: - Proven ability to **work together effectively** in c...
   Score: 0.4200
   Status: Słabe dopasowanie 🔸

3. Requirement: - Proven experience with various piping materials (e.g., copper, PVC, PEX). - Ab...
   Best Match: Personal and Team Skills: - Proven ability to **work together effectively** in c...
   Score: 0.5432
   Status: Słabe dopasowanie 🔸

4. Requirement: Soft Skills & Team Requirements: - A strong sense of **physical endurance** and ...
   Best Match: Personal and Team Skills: - Proven ability to **work

In [82]:
sbert_score_medium = sbert_pipeline(job_offers['medium']['text'], cv_candidate)
print('\n=== MEDIUM MATCH REPORT ===')
print(f"Overall Score: {sbert_score_medium['overall_score']:.4f}")
print(f"Expected Score Range: {job_offers['medium']['score']}")
print("\nBreakdown:")
for i, item in enumerate(sbert_score_medium['breakdown'], 1):
    print(f"\n{i}. Requirement: {item['requirement'][:80]}...")
    print(f"   Best Match: {item['best_match_in_cv'][:80]}...")
    print(f"   Score: {item['score']:.4f}")
    print(f"   Status: {item['status']}")


=== MEDIUM MATCH REPORT ===
Overall Score: 0.5534
Expected Score Range: 40-60

Breakdown:

1. Requirement: JOB OFFER: Junior Front-End Web Developer We are looking for a creative Junior F...
   Best Match: Personal and Team Skills: - Proven ability to **work together effectively** in c...
   Score: 0.3848
   Status: Słabe dopasowanie 🔸

2. Requirement: Required Technical Skills: - Proficiency in core web technologies: **HTML5, CSS3...
   Best Match: Technical Expertise: - **Expert level coding** in Python, used for ETL and compl...
   Score: 0.5495
   Status: Słabe dopasowanie 🔸

3. Requirement: - Knowledge of UI/UX principles and responsive design best practices. Soft Skill...
   Best Match: Personal and Team Skills: - Proven ability to **work together effectively** in c...
   Score: 0.6146
   Status: Dobre dopasowanie ⚠️

4. Requirement: - Proven ability to **collaborate effectively** with design and back-end teams. ...
   Best Match: Personal and Team Skills: - Proven ability to **

In [83]:
sbert_score_medium

{'overall_score': 0.5534,
 'breakdown': [{'requirement': 'JOB OFFER: Junior Front-End Web Developer We are looking for a creative Junior Front-End Web Developer to assist in building and maintaining our client-facing applications. Required Technical Skills: - Proficiency in core web technologies: **HTML5, CSS3, and JavaScript**. - Familiarity with a modern front-end framework (e.g., React or Vue.js).',
   'best_match_in_cv': 'Personal and Team Skills: - Proven ability to **work together effectively** in cross-functional teams. - Highly **articulate** and able to clearly explain technical results to non-technical stakeholders. - Driven by a **desire for knowledge** and continuous improvement.',
   'score': 0.38478192687034607,
   'status': 'Słabe dopasowanie 🔸'},
  {'requirement': 'Required Technical Skills: - Proficiency in core web technologies: **HTML5, CSS3, and JavaScript**. - Familiarity with a modern front-end framework (e.g., React or Vue.js). - Basic experience querying data vi

In [84]:
# perfect
sbert_score_perfect = sbert_pipeline(job_offers['perfect']['text'], cv_candidate)
print('\n=== PERFECT MATCH REPORT ===')
print(f"Overall Score: {sbert_score_perfect['overall_score']:.4f}")
print(f"Expected Score Range: {job_offers['perfect']['score']}")
print("\nBreakdown:")
for i, item in enumerate(sbert_score_perfect['breakdown'], 1):
    print(f"\n{i}. Requirement: {item['requirement'][:80]}...")
    print(f"   Best Match: {item['best_match_in_cv'][:80]}...")
    print(f"   Score: {item['score']:.4f}")
    print(f"   Status: {item['status']}")


=== PERFECT MATCH REPORT ===
Overall Score: 0.7034
Expected Score Range: 90-100

Breakdown:

1. Requirement: JOB OFFER: Data Science Intern / Junior Data Analyst We are seeking an enthusias...
   Best Match: CANDIDATE PROFILE: Data Science Graduate Summary: Enthusiastic graduate with a p...
   Score: 0.5551
   Status: Dobre dopasowanie ⚠️

2. Requirement: Required Technical Skills: - **Expert level coding** proficiency in **Python** f...
   Best Match: Technical Expertise: - **Expert level coding** in Python, used for ETL and compl...
   Score: 0.7745
   Status: Świetne dopasowanie ✅

3. Requirement: - Successful application of **statistical methods** for generating business insi...
   Best Match: Technical Expertise: - **Expert level coding** in Python, used for ETL and compl...
   Score: 0.7731
   Status: Świetne dopasowanie ✅

4. Requirement: - Highly **articulate** and able to clearly explain complex technical results to...
   Best Match: Personal and Team Skills: - Proven ability

# Join and final test

In [85]:
def calculate_final_score(job_offer: str, cv: str, sbert_model: SentenceTransformer,
                          nlp: Language, tfidf_vectorizer: TfidfVectorizer,
                          alpha = 0.7) -> Dict[str, any]:
    # Get scores and explaination tf-idf
    tfidf_clean = lemmatize_and_clean_texts([job_offer, cv], nlp)
    tfidf_matrix = tfidf_vectorizer.fit_transform(tfidf_clean)
    tfidf_similarity = cosine_similarity(
        tfidf_matrix[0:1],
        tfidf_matrix[1:2]
    )[0][0]
    explaination_tfidf = explain_tf_idf(tfidf_vectorizer, tfidf_matrix)

    # Get scores and explaination sbert
    sbert_chunks = clean_texts_for_sbert(job_offer, cv, nlp)
    sbert_report = calculate_precise_match(sbert_chunks)

    score_final = alpha * sbert_report['overall_score'] + (1 - alpha) * tfidf_similarity
    return {
        "final_score": round(score_final, 4),
        "sbert_report": sbert_report,
        "tfidf_similarity": round(tfidf_similarity, 4),
        "explaination_tfidf": explaination_tfidf
    }

In [86]:
final_result_poor = calculate_final_score(
    job_offers['poor']['text'], cv_candidate,
    SentenceTransformer(SBERT_MODEL), spacy.load(SPACY_MODEL),
    TfidfVectorizer(stop_words='english')
)
final_result_poor

{'final_score': np.float64(0.3573),
 'sbert_report': {'overall_score': 0.4664,
  'breakdown': [{'requirement': 'JOB OFFER: Certified Commercial Plumbing Technician We are seeking a reliable and physically capable Certified Commercial Plumbing Technician to join our field service team. Required Technical Skills: - Current state certification in commercial plumbing installation and repair.',
    'best_match_in_cv': 'Personal and Team Skills: - Proven ability to **work together effectively** in cross-functional teams. - Highly **articulate** and able to clearly explain technical results to non-technical stakeholders. - Driven by a **desire for knowledge** and continuous improvement.',
    'score': 0.35376501083374023,
    'status': 'Słabe dopasowanie 🔸'},
   {'requirement': 'Required Technical Skills: - Current state certification in commercial plumbing installation and repair. - Expertise in using and maintaining standard plumbing tools (e.g., pipe cutters, soldering torches). - Proven e

In [87]:
final_score_medium = calculate_final_score(
    job_offers['medium']['text'], cv_candidate,
    SentenceTransformer(SBERT_MODEL), spacy.load(SPACY_MODEL),
    TfidfVectorizer(stop_words='english')
)
final_score_medium

{'final_score': np.float64(0.4319),
 'sbert_report': {'overall_score': 0.5534,
  'breakdown': [{'requirement': 'JOB OFFER: Junior Front-End Web Developer We are looking for a creative Junior Front-End Web Developer to assist in building and maintaining our client-facing applications. Required Technical Skills: - Proficiency in core web technologies: **HTML5, CSS3, and JavaScript**. - Familiarity with a modern front-end framework (e.g., React or Vue.js).',
    'best_match_in_cv': 'Personal and Team Skills: - Proven ability to **work together effectively** in cross-functional teams. - Highly **articulate** and able to clearly explain technical results to non-technical stakeholders. - Driven by a **desire for knowledge** and continuous improvement.',
    'score': 0.38478192687034607,
    'status': 'Słabe dopasowanie 🔸'},
   {'requirement': 'Required Technical Skills: - Proficiency in core web technologies: **HTML5, CSS3, and JavaScript**. - Familiarity with a modern front-end framework (e

In [88]:
final_result_perfect = calculate_final_score(
    job_offers['perfect']['text'], cv_candidate,
    SentenceTransformer(SBERT_MODEL), spacy.load(SPACY_MODEL),
    TfidfVectorizer(stop_words='english')
)
final_result_perfect

{'final_score': np.float64(0.6616),
 'sbert_report': {'overall_score': 0.7034,
  'breakdown': [{'requirement': 'JOB OFFER: Data Science Intern / Junior Data Analyst We are seeking an enthusiastic and technically proficient graduate to join our Data Science team for a junior role or internship. Required Technical Skills: - **Expert level coding** proficiency in **Python** for data manipulation and analysis.',
    'best_match_in_cv': 'CANDIDATE PROFILE: Data Science Graduate Summary: Enthusiastic graduate with a passion for transforming complex data into actionable insights and analysing). Technical Expertise: - **Expert level coding** in Python, used for ETL and complex calculations.',
    'score': 0.5551204085350037,
    'status': 'Dobre dopasowanie ⚠️'},
   {'requirement': 'Required Technical Skills: - **Expert level coding** proficiency in **Python** for data manipulation and analysis. - Deep knowledge of relational databases, with proven experience managing data using **Structured Q

# Explainability

In [51]:
def explain_tf_idf(tfidf_vectorizer: TfidfVectorizer, matrix: csr_matrix):
    feature_names = tfidf_vectorizer.get_feature_names_out()
    dense = matrix.todense()

    denselist = dense.tolist()
    df = pd.DataFrame(denselist, columns=feature_names, index=['job_offer', 'cv_candidate'])

    common_words = df.loc['job_offer'] * df.loc['cv_candidate']
    common_words = common_words[common_words > 0].sort_values(ascending=False)

    return common_words



In [52]:
common_words_poor = explain_tf_idf(tfidf_vec_poor, matrix_poor)
print('\nCommon words TF-IDF poor (Lemmatized, no N-grams):')
print(common_words_poor)


Common words TF-IDF poor (Lemmatized, no N-grams):
team          0.022796
technical     0.017097
skills        0.011398
ability       0.011398
use           0.011398
experience    0.005699
prove         0.005699
project       0.005699
expertise     0.005699
work          0.005699
dtype: float64


In [53]:
common_words_medium = explain_tf_idf(tfidf_vec_medium, matrix_medium)
print('\nCommon words TF-IDF medium (Lemmatized, no N-grams):')
print(common_words_medium)


Common words TF-IDF medium (Lemmatized, no N-grams):
datum          0.022836
team           0.022836
knowledge      0.022836
technical      0.017127
skills         0.011418
use            0.011418
effectively    0.005709
desire         0.005709
ability        0.005709
experience     0.005709
manage         0.005709
query          0.005709
sql            0.005709
dtype: float64


In [54]:
common_words_perfect = explain_tf_idf(tfidf_vec_perfect, matrix_perfect)
print('\nCommon words TF-IDF perfect (Lemmatized, no N-grams):')
print(common_words_perfect)


Common words TF-IDF perfect (Lemmatized, no N-grams):
technical       0.046353
team            0.046353
knowledge       0.030902
datum           0.030902
data            0.030902
science         0.023177
graduate        0.015451
experience      0.015451
prove           0.015451
skills          0.015451
database        0.015451
complex         0.015451
use             0.015451
desire          0.007726
deep            0.007726
continuous      0.007726
cross           0.007726
able            0.007726
articulate      0.007726
clearly         0.007726
cod             0.007726
ability         0.007726
hands           0.007726
functional      0.007726
effectively     0.007726
enthusiastic    0.007726
expert          0.007726
explain         0.007726
drive           0.007726
method          0.007726
manage          0.007726
level           0.007726
language        0.007726
insight         0.007726
highly          0.007726
query           0.007726
postgre         0.007726
relational      0.00

# Final object

In [ ]:
# find a perfect class name for this file
class SimilarityCheck:
	"""Encapsulate similarity checks and explainability helpers."""
	